In [1]:
import os, sys, email, datetime, pprint, re, time, html, pickle
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *

pd.options.display.max_colwidth = 1000

In [34]:
with open('/Users/taroaso/myprojects/OpenIE/trec/output/tagme.pickle', mode="rb") as f:
    tagme_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/ner.pickle', mode="rb") as f:
     ner_result = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/type_translation.pickle', mode="rb") as f:
     translation = pickle.load(f)

In [3]:
from entityLinking import getWikidataID

In [4]:
for result in tagme_result["003b01c10c55$f0f62060$f3468ed5@huftis"]['annotations']:
    try:
        if result['rho'] >= 0.3:
            res = getWikidataID(result['id'])
            pprint.pprint(result)
            pprint.pprint(res)
    except:
        pass

{'end': 268,
 'id': 839715,
 'link_probability': 1,
 'rho': 0.7779867649078369,
 'spot': 'numeric entity',
 'start': 254,
 'title': 'Numeric character reference'}
{'batchcomplete': '',
 'query': {'pages': {'839715': {'ns': 0,
                                'pageid': 839715,
                                'pageprops': {'defaultsort': 'Numeric '
                                                             'Character '
                                                             'Reference',
                                              'wikibase_item': 'Q1204079'},
                                'title': 'Numeric character reference'}}}}
{'end': 287,
 'id': 59126,
 'link_probability': 0.37846481800079346,
 'rho': 0.3699946403503418,
 'spot': 'apostrophe',
 'start': 277,
 'title': 'Apostrophe'}
{'batchcomplete': '',
 'query': {'pages': {'59126': {'ns': 0,
                               'pageid': 59126,
                               'pageprops': {'wikibase-shortdesc': 'Punctuation '
  

In [58]:
ner_result[uid]

[{'spot': 'doc', 'start': 21, 'end': 24, 'type': 'ORG'},
 {'spot': '<', 'start': 109, 'end': 110, 'type': 'NORP'},
 {'spot': 'xhtml', 'start': 177, 'end': 182, 'type': 'ORG'},
 {'spot': 'xhtml', 'start': 237, 'end': 242, 'type': 'ORG'},
 {'spot': 'xhtml', 'start': 342, 'end': 347, 'type': 'ORG'},
 {'spot': 'javascript', 'start': 352, 'end': 362, 'type': 'PERSON'},
 {'spot': 'two', 'start': 427, 'end': 430, 'type': 'CARDINAL'},
 {'spot': 'john russell ve3ll@rac.ca',
  'start': 439,
  'end': 464,
  'type': 'PERSON'},
 {'spot': "L's", 'start': 476, 'end': 479, 'type': 'ORG'}]

In [124]:
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
BASE = Namespace("http://www.kde.cs.tsukuba.ac.jp/~aso/w3c-email/")
#EMAIL = Namespace("http://www.w3.org/2000/10/swap/pim/email#")
OLIA = Namespace("http://purl.org/olia/olia.owl#")
NERD = Namespace("http://nerd.eurecom.fr/ontology#")
NIF = Namespace("http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#")
ITSRDF = Namespace("https://www.w3.org/2005/11/its/rdf#")
WD = Namespace("http://www.wikidata.org/entity/")

In [125]:
# RDFグラフ
g = Graph()
g.bind('',BASE)
g.bind('schema',SDO)
g.bind('foaf',FOAF)
g.bind('owl',OWL)
g.bind('rdf',RDF)
g.bind('rdfs',RDFS)
g.bind('olia',OLIA)
g.bind('nerd',NERD)
g.bind('nif',NIF)
g.bind('itsrdf',ITSRDF)
g.bind('wd',WD)

In [126]:
def body2rdf(g,el_result,ner_result): #g:RDFグラフ，el_result:tagmeの結果(json)のKey,Valueのtuple, ner_result:nerの結果
    uid = el_result[0]
    annotations = el_result[1]['annotations']

    emailmessage = re.sub(r'[^a-zA-Z_0-9]','_',uid) #URI参照される表記に変換する

    for annotation in annotations:
        if annotation['link_probability'] >= 0.3:
            spot = annotation['spot']
            start = annotation['start']
            end = annotation['end']
            offset_based_string = emailmessage + '#offset_' + str(start) + '_' + str(end)
            # Entity, Mentionに共通する記述
            g.add((BASE[emailmessage],SDO.mentions,BASE[offset_based_string]))
            g.add((BASE[offset_based_string],RDF.type,NIF.String))
            g.add((BASE[offset_based_string],NIF.beginIndex,Literal(start,datatype=XSD.integer)))
            g.add((BASE[offset_based_string],NIF.endIndex,Literal(end,datatype=XSD.integer)))
            g.add((BASE[offset_based_string],NIF.isString,Literal(spot,lang='en')))
            # Entity
            if 'rho' in annotation:
                rho = annotation['rho']
                pid = annotation['id']
                # 閾値0.3以上のEntityを記述する
                if annotation['rho'] >= 0.3:
                    res = getWikidataID(pid)
                    try:
                        wid = res['query']['pages'][str(pid)]['pageprops']['wikibase_item']
                        g.add((BASE[offset_based_string],ITSRDF.taIdentRef,WD[wid]))
                        g.add((BASE[offset_based_string],ITSRDF.taConfidence,Literal(rho,datatype=XSD.decimal)))
                    except KeyError: #getWikidataIDの結果がなかった場合（['pageprops']のキーがない）はMentionの記述をする
                        link_probability = annotation['link_probability']
                        g.add((BASE[offset_based_string],RDF.type,NIF.EntityOccurrence))
                        g.add((BASE[offset_based_string],NIF.entityOccurrenceConf,Literal(link_probability,datatype=XSD.decimal)))
                        for ner in ner_result[uid]:
                            if ner['spot'] == spot:
                                spacy_type = ner['type']
                                schema_type = translation[spacy_type]
                                g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type]))
                            else:
                                schema_type = 'Thing'
                                g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type])) 
                # 閾値0.3未満のEntityはMentionと同じ記述
                else:
                    link_probability = annotation['link_probability']
                    g.add((BASE[offset_based_string],RDF.type,NIF.EntityOccurrence))
                    g.add((BASE[offset_based_string],NIF.entityOccurrenceConf,Literal(link_probability,datatype=XSD.decimal)))
                    for ner in ner_result[uid]:
                        if ner['spot'] == spot:
                            spacy_type = ner['type']
                            schema_type = translation[spacy_type]
                            g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type]))
                        else:
                            schema_type = 'Thing'
                            g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type]))    
            # Mention
            else:
                link_probability = annotation['link_probability']
                g.add((BASE[offset_based_string],RDF.type,NIF.EntityOccurrence))
                g.add((BASE[offset_based_string],NIF.entityOccurrenceConf,Literal(link_probability,datatype=XSD.decimal)))
                for ner in ner_result[uid]:
                    if ner['spot'] == spot:
                        spacy_type = ner['type']
                        schema_type = translation[spacy_type]
                        g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type]))
                    else:
                        schema_type = 'Thing'
                        g.add((BASE[offset_based_string],ITSRDF.taClassRef,SDO[schema_type]))

In [127]:
results = list(tagme_result.items())

In [128]:
total_start = time.time()
for result in results:
    part_start = time.time()
    if result[1] is None:
        pass
    else:
        body2rdf(g,result,ner_result)
    print(time.time()-part_start)
print(time.time()-total_start)

2.1457672119140625e-06
4.607988119125366
2.751190185546875
7.944271087646484
13.289409875869751
1.2093942165374756
14.0724937915802
72.1034288406372
4.292540788650513
7.860082149505615
6.124112129211426
1.8525199890136719
13.693905115127563
1.1610238552093506
1.2662639617919922
0.5884590148925781
2.5013909339904785
6.010793924331665
21.38880205154419
9.5367431640625e-07
9.5367431640625e-07
1.2301502227783203
1.1920928955078125e-06
6.136675834655762
4.1937127113342285
7.276799917221069
1.2135188579559326
10.247652053833008
1.203444004058838
10.432897090911865
18.9549777507782
35.22776770591736
0.6319637298583984
4.267823219299316
9.5367431640625e-07
3.0267789363861084
12.947268962860107
4.243364095687866
1.5020370483398438e-05
0.5872843265533447
7.063766002655029
0.0
2.434718132019043
4.420705080032349
2.4162752628326416
21.436065673828125
1.7999861240386963
4.326037883758545
1.1920928955078125e-06
11.872135877609253
0.6663990020751953
9.5367431640625e-07
17.132312774658203
4.8446133136

In [129]:
# turtle形式でファイル出力
g.serialize(destination='/Users/taroaso/myprojects/OpenIE/trec/output/mail_body_entity.ttl', format='turtle')
#print(g.serialize(format="turtle").decode("utf-8"))